<a href="https://colab.research.google.com/github/ssnirgudkar/Datasetpaper-final/blob/main/IR_7classes_Unet_Finalpaper_submission_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this as the final code. - DOES NOT USE ANY PREEXISTING WEIGHTS

In [1]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

In [2]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/test.zip", "r")
zip_ref.extractall("/content/IRDatasetFinal")
zip_ref.close()

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/testannot.zip", "r")
zip_ref.extractall("/content/IRDatasetFinal")
zip_ref.close()

'''
# Zip file with handpicked test images
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/select_test_images.zip", "r")
zip_ref.extractall("/content/IRDatasetFinal")
zip_ref.close()

# Zip file with masks of handpicked test images
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/select_test_masks.zip", "r")
zip_ref.extractall("/content/IRDatasetFinal")
zip_ref.close()

# copy the select_test_images and masks in test and testannot respectively 
!cp /content/IRDatasetFinal/select_test_images/*.png /content/IRDatasetFinal/test
!cp /content/IRDatasetFinal/select_test_masks/*.png /content/IRDatasetFinal/testannot
'''

'\n# Zip file with handpicked test images\nzip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/select_test_images.zip", "r")\nzip_ref.extractall("/content/IRDatasetFinal")\nzip_ref.close()\n\n# Zip file with masks of handpicked test images\nzip_ref = zipfile.ZipFile("/content/drive/MyDrive/IRDatasetConsolidated/select_test_masks.zip", "r")\nzip_ref.extractall("/content/IRDatasetFinal")\nzip_ref.close()\n\n# copy the select_test_images and masks in test and testannot respectively \n!cp /content/IRDatasetFinal/select_test_images/*.png /content/IRDatasetFinal/test\n!cp /content/IRDatasetFinal/select_test_masks/*.png /content/IRDatasetFinal/testannot\n'

In [4]:
import os 
APP_FOLDER = '/content/IRDatasetFinal/test'
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1
   

print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Searching in :  /content/IRDatasetFinal/test
Total number of files 4508
Total Number of directories 0
Total: 4508


In [ ]:
'''
# Now remove images that are not pure (rotated, mirrored etc from the test and testannot folders) 
# as we do not want to use them for testing 
import os
import sys
import shutil

test_dir = "/content/IRDatasetFinal/test"
test_mask_dir = "/content/IRDatasetFinal/testannot"
# Move rotated & mirrored files into tmp directory
# The format of these files is aXXX_XXX_XX.png and aXXX_XXX_XX_XX.png
# Number of underscores is 2 or 3
dest_test_dir = "/content/IRDatasetFinal/test_tmp"
dest_mask_dir = "/content/IRDatasetFinal/testannot_tmp"

# Move non-augmented files from test directory into a temporary directory
if (False == os.path.exists(dest_test_dir)):
  os.mkdir(dest_test_dir)
if (False == os.path.exists(dest_mask_dir)):
  os.mkdir(dest_mask_dir)


for eachFile in os.listdir(test_dir):
  numUnderscores = eachFile.count('_')
  if (numUnderscores > 1):
    # File names in image & mask directories are identical so move both files in one shot!
    shutil.move(os.path.join(test_dir, eachFile), os.path.join(dest_test_dir))
    shutil.move(os.path.join(test_mask_dir, eachFile), os.path.join(dest_mask_dir))
    
'''
    for eachFile in os.listdir(test_mask_dir):
      numUnderscores = eachFile.count('_')
      if (numUnderscores > 1):
        # File names in image & mask directories are identical so move both files in one shot!
        shutil.move(os.path.join(test_mask_dir, eachFile), os.path.join(dest_mask_dir))
'''
'''

In [ ]:
'''
# Count the number of images in test and testannot again
import os 
APP_FOLDER = '/content/IRDatasetFinal/test'
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1
   

print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))
'''

# Image segmentation with a U-Net-like architecture

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2019/03/20<br>
**Last modified:** 2020/04/20<br>
**Description:** Image segmentation model trained from scratch on the Oxford Pets dataset.

##Inference corresponding to run#7 of training - No preloaded weights - actual size images + mirror + rotate . GPU (Standard RAM)
train - 26081  val - 7452  test - 3726 
, epochs = 100
training time - 2 days, testing -  min  
Model - /Models/Unet-7classes-finalpapersubmission2
file names prefix - run11 
Loss - 0.0341




In [5]:
import tensorflow as tf
print("keras version is", tf.keras.__version__)
print ("tf version is", tf.__version__) 
!python --version

keras version is 2.7.0
tf version is 2.7.0
Python 3.7.12


## Prepare paths of input images and target segmentation masks

In [6]:
import os
import numpy as np
import cv2

input_dir_test = "/content/IRDatasetFinal/test" # directory containing 2570 IR input images
target_dir_test = "/content/IRDatasetFinal/testannot" # directory containing 2570 segmented IR images

img_size = (512, 640) #it's height and width. it's a 1/2 sized image from the original image from segment.ai
#img_size = (512, 640) #it's height and width. This is the original image from segment.ai
#img_size_width_ht = (320,256)
img_size_width_ht = (640,512)

num_classes = 7
batch_size = 10
test_batch_size = 1
 

# check for distinct label values in all masked files 
def getFullyQualifiedImagePaths(image_dir):
  return sorted([ os.path.join(image_dir, fname)
                  for fname in os.listdir(image_dir)
                  if fname.endswith(".png")
                ])
 


In [7]:
# Inference 
#prints how many files are in the test batch
input_test_paths = getFullyQualifiedImagePaths(input_dir_test)
target_test_paths = getFullyQualifiedImagePaths(target_dir_test)

#prints how many files are in the batch
print("Number of test images:", len(input_test_paths))
print("Number of test masks:", len(target_test_paths))

#pattern of the test images and masks 
for input_path, target_path in zip(input_test_paths[:5], target_test_paths[:5]):
    print(input_path, "|", target_path)

#!nvidia-smi

Number of test images: 4508
Number of test masks: 4508
/content/IRDatasetFinal/test/a1570555926_835540.png | /content/IRDatasetFinal/testannot/a1570555926_835540.png
/content/IRDatasetFinal/test/a1570555926_835540_2.png | /content/IRDatasetFinal/testannot/a1570555926_835540_2.png
/content/IRDatasetFinal/test/a1570555926_835540_412.png | /content/IRDatasetFinal/testannot/a1570555926_835540_412.png
/content/IRDatasetFinal/test/a1570555926_835540_412_2.png | /content/IRDatasetFinal/testannot/a1570555926_835540_412_2.png
/content/IRDatasetFinal/test/a1570555926_835540_415.png | /content/IRDatasetFinal/testannot/a1570555926_835540_415.png


## Prepare `Sequence` class to load & vectorize batches of data

In [8]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            #in case we want to see if masked image have the right values
            #print(tensorflow.keras.preprocessing.image.img_to_array(img))
            y[j] = np.expand_dims(img, 2)
            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2: 
            #if classes are 3, keras expects masked values to be 0,1,2 only. Cannot take 1,3,5. It checks numerically not just the number of masked values
            #y[j] += 1 
        return x, y
#!nvidia-smi


In [9]:
# Inference 
import random
# Instantiate data Sequences for each split

test_gen = OxfordPets(test_batch_size, img_size, input_test_paths, target_test_paths)
#!nvidia-smi

### Approach
1. load a model 
2. in a for loop
3. OxfordPets give 2 images at a time
4. run prediction
5. visualize and save

## Visualize predictions

In [ ]:
#!nvidia-smi



In [10]:
# Load Model 
#reconstructed_model1 = keras.models.load_model("/content/drive/MyDrive/Models/Unet-7classes-finalpapersubmission2") # model with 2019 and 2020 data for 100 epochs
#reconstructed_model1 = keras.models.load_model("/content/drive/MyDrive/Models/unet/Unet-7classes-finalpapersubmission_2020only") # model with 2020 only images for 50 epochs
reconstructed_model1 = keras.models.load_model("/content/drive/MyDrive/Models/unet/Unet-7classes-finalpapersubmission_2019&2020_shuffled") # model with all imaged and shuffled



In [ ]:
#### Prediction

from IPython.display import Image, display
import PIL
from PIL import ImageOps

def display_mask(i,file_num_dir,imagefilepathtokens):
    """Quick utility to display a model's prediction. we need color masked images, so displaying both color and black and white"""
    
    #0 - sky(dark.blue). 
    #1 - water(light.blue).   
    #2 - bridge(yellow).   
    #3 - obstacle(purple).  
    #4- living ob(green).  
    #5- backgnd (orange). 
    #6 - self(pink)
    # Colors are same as segments.ai scheme
    label_colours = [(0,113,188), (216,82,24), (236,176,31), (125, 46, 141), (118, 171, 47), (161, 19, 46), (255,0,0)]  
    
    
    mask = np.argmax(test_preds1[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    #print("mask shape = ", np.shape(mask))
    #print("mask length = ", len(mask[i, 0]))
    #print("mask length = ", len(mask[i]))
    #print("Unique pixel values = ", np.unique(mask))
    #print("Type of mask = ", type(mask))
    
   
    #img1 = PIL.Image.new('RGB', (640, 512))
    img1 = PIL.Image.new('RGB', img_size_width_ht)
    #print ("image size", img_size_width_ht)
    pixels = img1.load()
    #print(type(pixels))
    #print(pixels[0,0])
    for j_, j in enumerate(mask[:, :, 0]):
        #print (j_, j)
        for k_, k in enumerate(j):
             #print(k_, k)
             if k < num_classes:
                pixels[k_,j_] = label_colours[k]
    output = np.array(img1)

    outputpicturePath = '/content/drive/MyDrive/InferenceLarge20192020shuffled/unet/picture' + '/' + str(file_num_dir) + '--runreviseddata-1--' + imagefilepathtokens[-1]     
    
    #print("this is the colored inferred image")
    #display(img1)

    ##Saving the category ids in an image for programatic IoU check 
    img1.save(outputpicturePath)

    '''
    print("this is the gray inferred image")
    img = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)
    '''
    return mask
   



test_input_dir1 = "/content/IRDatasetFinal/test"  
test_target_input_dir1 = "/content/IRDatasetFinal/testannot"  
test_input_img_paths1 = getFullyQualifiedImagePaths(test_input_dir1)
test_target_img_paths1 = getFullyQualifiedImagePaths(test_target_input_dir1)
no_of_test_img = len(test_input_img_paths1)
print("Num test images = {0}".format(no_of_test_img))

prev = 0
for i in range(1, no_of_test_img+1, 2):
#for i in range(11, 20+1, 2):
    #print("prev={0},i={1}".format(prev,i))
    test_gen1 = OxfordPets(test_batch_size, img_size, test_input_img_paths1[prev:i+1], test_target_img_paths1[prev:i+1])
    print("iteration running is", i)
    #print("image_path", test_input_img_paths1[i])
    #!nvidia-smi
    tf.compat.v1.GPUOptions(allow_growth=True)
    test_preds1 = reconstructed_model1.predict(test_gen1)
    #print("Size of test_preds1 = {0}".format(len(test_preds1)))
    #!nvidia-smi
    #free memory / restart runtime 

    # Inference 
    ## Now visualize predictions for a specific image. Change the value of i

    for jj in range(0, 2):
        # check if ground-truth target mask as any object - class id - 3
        image_gt = tf.keras.preprocessing.image.load_img(test_target_img_paths1[jj])
        input_arr = tf.keras.preprocessing.image.img_to_array(image_gt)
        gt_mask_array = np.unique(input_arr) 
        object_living_array = [1] # we need all 300+ images. hence putting here 1 instead of 3,4
        #print(gt_mask_array)
        #intersect_array_object = np.intersect1d(gt_mask_array, object_array) 
        #intersect_array_living = np.intersect1d(gt_mask_array, living_array) 
        if((set(object_living_array) & set(gt_mask_array))== set(object_living_array)):
            #print ("intersect output 2", gt_mask_array )
            #print(gt_mask_array)
            #display the raw input image 

            file_num_dir = prev+jj #this denotes the sequence of the image in the test dir
            #print("this is the raw image")
            #display(Image(filename=test_input_img_paths1[file_num_dir]))
          
            #display the ground truth masked image 
            img = PIL.ImageOps.autocontrast(load_img(test_target_img_paths1[file_num_dir]))
            #print("this is the ground truth mask")
            #display(img)
          
            #set file names for storing the prediction 
            imagefilepathtokens = test_input_img_paths1[file_num_dir].split('/')
            #print("tokens = ", imagefilepathtokens)

            outputFilePath = '/content/drive/MyDrive/InferenceLarge20192020shuffled/unet/program' + '/' + str(file_num_dir) + '--runreviseddata-1--' + imagefilepathtokens[-1] 
            #print("output file name = ", outputFilePath)

            #invoke the prediction function 
            output_mask = display_mask(jj,file_num_dir,imagefilepathtokens)  # Note that the model only sees inputs at 150x150.
            output_mask = np.squeeze(output_mask, axis=2)
            #result = np.where(output_mask==0)
            #print("indices where output_mask has 0 value = ", result)
            output_Im = PIL.Image.fromarray(output_mask.astype(np.uint8))

            ##Saving the category ids in an image for programatic IoU check 
            output_Im.save(outputFilePath)      
    prev = i+1      



